Read Mesh

In [1]:
from nassu.lnas import LagrangianFormat
import pathlib

mesh = LagrangianFormat.from_file(pathlib.Path("./output/simulation_test/lnas/G200/G200.lnas"))

Read historic series

In [2]:
import pandas as pd

data_path = pathlib.Path("./output/simulation_test/probes/hist_series/series1")

body_hist_series = pd.read_hdf(data_path / "bodies.G200.data.h5")
sfc1_data = pd.read_hdf(data_path / "bodies.m1_inferior.data.h5")
sfc2_data = pd.read_hdf(data_path / "bodies.m2_inferior.data.h5")
sfc3_data = pd.read_hdf(data_path / "bodies.m3_inferior.data.h5")

last_index = body_hist_series.point_idx.max()
sfc1_data.point_idx += last_index + 1

last_index = sfc1_data.point_idx.max()
sfc2_data.point_idx += last_index + 1

last_index = sfc2_data.point_idx.max()
sfc3_data.point_idx += last_index + 1

len(body_hist_series.index) + len(sfc1_data.index) + len(sfc2_data.index) + len(sfc3_data.index)

174184

Offset surface

In [3]:
import numpy as np

NORMAL_OFFSET = -0.125


for surface_name in ["m1_main", "m2_main", "m3_main"]:
    tri_normals = mesh.geometry_from_surface(surface_name).normals.copy()
    tri_verts = mesh.geometry_from_surface(surface_name).triangle_vertices.copy()
    tri_verts += np.repeat(tri_normals, 3, axis=0).reshape(-1, 3, 3) * NORMAL_OFFSET

    new_verts = tri_verts.copy().reshape(-1, 3)
    new_verts = np.unique(new_verts, axis=0)

    new_triangles = []

    for tri in tri_verts:
        tri_index = []
        for v in tri:
            matches = np.all(new_verts == v, axis=1)
            indices = np.where(matches)
            tri_index.append(indices[0][0])
        tri_index.reverse()
        new_triangles.append(np.array(tri_index))

    new_triangles = np.array(new_triangles, dtype=np.uint32) + len(mesh.geometry.vertices)

    new_surface_name = surface_name.split("_")[0] + "_inferior"
    new_tri_indices = np.arange(
        mesh.geometry.triangles.shape[0], mesh.geometry.triangles.shape[0] + new_triangles.shape[0]
    )
    mesh.surfaces[new_surface_name] = new_tri_indices

    mesh.geometry.vertices = np.concatenate((mesh.geometry.vertices, new_verts))
    mesh.geometry.triangles = np.concatenate((mesh.geometry.triangles, new_triangles))

    mesh.geometry._update_triangles_vertices()
    mesh.geometry._update_normals()
    mesh.geometry._update_areas()
    mesh.geometry._update_vertices_normals()


filename = pathlib.Path("./output/simulation_test/lnas/G200/G200.merged.lnas")
export_path = pathlib.Path("./output/simulation_test/lnas/G200/G200.merged.stl")

mesh.to_file(filename)
read_mesh = LagrangianFormat.from_file(filename)

read_mesh.geometry.triangles.shape, mesh.geometry.triangles.shape

((21773, 3), (21773, 3))

Join historic series

In [4]:
new_df = pd.concat([body_hist_series, sfc1_data, sfc2_data, sfc3_data]).sort_values(
    by=["time_step", "point_idx"]
)
new_df.to_hdf(data_path / "bodies.G200.merged.data.h5", key="df", mode="w")

new_df

,point_idx,rho,ux,uy,uz,time_step
0,0,0.985114,1.346108e-04,0.000040,0.000119,1000.0
1,1,0.985309,2.115229e-04,-0.000032,0.000112,1000.0
2,2,0.985323,6.648335e-05,0.000076,0.000031,1000.0
3,3,0.985529,1.164332e-04,-0.000019,-0.000150,1000.0
4,4,0.985755,3.335405e-05,-0.000021,-0.000058,1000.0
...,...,...,...,...,...,...
21768,21768,1.002900,3.396856e-07,0.000009,0.000014,18500.0
21769,21769,1.002704,-1.329287e-05,0.000002,0.000029,18500.0
21770,21770,1.002752,1.921312e-05,0.000011,-0.000033,18500.0
21771,21771,1.002660,2.029974e-05,0.000016,-0.000010,18500.0
